In [1]:
import ctypes
import numpy as np
from lib import *

import pandas as pd
from numpy import genfromtxt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report
import json

In [2]:
def vector_to_class_label(vector):
    max_index = max(range(len(vector)), key=vector.__getitem__)
    return max_index 

vector = [0.03871405, 0.13871405, 1.03871405]
class_label = vector_to_class_label(vector)
print("Classe :", class_label)

Classe : 2


In [3]:
test_acappella = genfromtxt('data/acappella_test.csv', delimiter=',')
train_acappella = genfromtxt('data/acappella_train.csv', delimiter=',')
train_accoust = genfromtxt('data/accoust_train.csv', delimiter=',')
test_accoust = genfromtxt('data/accoust_test.csv', delimiter=',')
train_electro = genfromtxt('data/electro_train.csv', delimiter=',')
test_electro = genfromtxt('data/electro_test.csv', delimiter=',')

In [4]:
vec_aca = np.zeros(train_acappella.shape[0])
vec_aca=  vec_aca.reshape(train_acappella.shape[0], 1)
train_acappella = np.hstack((train_acappella,vec_aca ))

vec_acc = np.ones(train_accoust.shape[0])
vec_acc =  vec_acc.reshape(train_accoust.shape[0], 1)
train_accoust = np.hstack((train_accoust,vec_acc ))

vec_ele = np.ones(train_electro.shape[0])*2
vec_ele =  vec_ele.reshape(train_electro.shape[0], 1)
train_electro = np.hstack((train_electro,vec_ele ))

In [5]:
print(train_acappella.shape)
print(train_accoust.shape)
print(train_electro.shape)

(2418, 1026)
(2418, 1026)
(2418, 1026)


In [6]:
# contatenate train_acappella, train_accoust, train_electro as np array
train = np.concatenate((train_acappella, train_accoust, train_electro), axis=0)
data = train[:,0:train.shape[1]-1]
label = train[:,-1]
data = data/255


In [7]:
X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.2, random_state=42 ,stratify=label)

In [8]:
y_train = y_train.reshape(y_train.shape[0], 1)
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(y_train)
y_train = enc.transform(y_train).toarray()
y_train = np.where(y_train == 0, -1, y_train)

In [9]:
y_train

array([[-1.,  1., -1.],
       [-1.,  1., -1.],
       [-1.,  1., -1.],
       ...,
       [ 1., -1., -1.],
       [ 1., -1., -1.],
       [-1., -1.,  1.]])

In [10]:
all_training_inputs = np.float64(X_train.flatten())
all_training_expected_outputs = np.float64(y_train.flatten())

npl = np.array([X_train.shape[1],100,50, y_train.shape[1]])
npl_size = len(npl)

sample_inputs = np.array(list(X_train[0]))
print(sample_inputs)
sample_inputs_size = len(sample_inputs)

[0.00392157 0.         0.         ... 0.95294118 0.94117647 0.9372549 ]


In [11]:
print("npl: ", npl)
print("npl_size: ", npl_size)
print("X_train.shape: ", X_train.shape)
print("y_train.shape: ", y_train.shape)


npl:  [1025  100   50    3]
npl_size:  4
X_train.shape:  (5803, 1025)
y_train.shape:  (5803, 3)


In [12]:
model_pointer = create_mlp_model(npl, npl_size)

predict = predict_mlp_model(model_pointer, sample_inputs , sample_inputs_size, True , npl)
print(predict)
delete_float_array(predict, npl)
train_mlp_model(model_pointer, all_training_inputs, len(X_train), len(X_train[0]), all_training_expected_outputs, len(y_train[0]), 0.001, 100, True)
predict = predict_mlp_model(model_pointer, sample_inputs , sample_inputs_size, True , npl)
label_predict = vector_to_class_label(predict)

print(predict)
print("label_predict: ", label_predict)
#delete_mlp_model(model_pointer)
delete_float_array(predict, npl)

y_pred = []
for i in range(len(X_test)):
    predict = predict_mlp_model(model_pointer, X_test[i] , sample_inputs_size, True , npl)
    #label_predict = vector_to_class_label(predict)
    y_pred.append(predict)
    delete_float_array(predict, npl)
#target_names = ['acappella', 'accoust', 'electro']
#print(classification_report(y_test, y_pred, target_names=target_names))

#delete_mlp_model(model_pointer)

create_mlp_model
create_mlp_model done
[0.99199398 0.99199398 0.99199398]
train_mlp_model
train_mlp_model done
[0.95559793 0.97917607 0.97898924]
label_predict:  1


In [13]:
y_pred_label = np.argmax(y_pred, axis=1)
label_list = [0,1,2]
predicted_classes  = [label_list[label_index] for label_index in y_pred_label]
print(classification_report(y_test, predicted_classes))

              precision    recall  f1-score   support

         0.0       0.31      0.46      0.37       483
         1.0       0.33      0.23      0.27       484
         2.0       0.31      0.26      0.28       484

    accuracy                           0.32      1451
   macro avg       0.32      0.32      0.31      1451
weighted avg       0.32      0.32      0.31      1451



In [33]:
save_mlp_model(model_pointer, "test.mlp")
delete_mlp_model(model_pointer)

MLP model saved to file: test.mlp


In [2]:
model_pointer = load_mlp_model("test.mlp")

: 

: 